# Máquinas de Boltzman restringidas

In [1]:
# Importar las librerias
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [37]:
# importar el dataset
movies = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-1m/movies.dat',
                     sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1')
movies.head(3)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [3]:
movies.shape

(3883, 3)

In [38]:
users = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-1m/users.dat',
                     sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1')
users.head(3)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


In [39]:
ratings = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-1m/ratings.dat',
                     sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1')
ratings.head(3)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [40]:
# Preparar el conjunto de entrenamiento y de testing
training_set = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-100k/u1.base', sep = '\t')
training_set = np.array(training_set, dtype = 'int')

In [41]:
test_set = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%205%20-%20Boltzmann%20Machines%20(BM)/ml-100k/u1.test', sep = '\t')
test_set = np.array(test_set, dtype = 'int')

In [42]:
# Obtener el numero de usuarios y de peliculas
number_users = max(max(training_set[:, 0]), max(test_set[:, 0]))
number_movies = max(max(training_set[:, 1]), max(test_set[:, 1]))

print('El numero de usuarios es {}'.format(number_users))
print('El numero de peliculas es {}'.format(number_movies))

El numero de usuarios es 943
El numero de peliculas es 1682


In [43]:
# Convertir los datos en un array X[u, i] con usuarios u en fila y peliculas i en columna

def convert(data):
  new_data = []
  for id_user in range(1, number_users + 1):
    id_movies = data[:,  1 ][data[:, 0] == id_user ]
    id_ratings = data[:,  2 ][data[:, 0] == id_user ]
    ratings = np.zeros(number_movies)
    ratings[id_movies - 1] = id_ratings 
    new_data.append(list(ratings))
  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [45]:
# Convertir los datos a tensores de Torch

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [46]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [47]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [48]:
# Convertir las valoraciones a valores binarios 1 (Me gusta) 0 (No me gusta)
training_set[training_set == 0] = -1 # -1 no ha valorado
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1 # -1 no ha valorado
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1


### Crear la arquitectura de la Red neuronal (Modelo probabilistico grafico)

In [49]:
class RBM():
  def __init__(self, visible_nodes, hidden_nodes):
    self.weights = torch.randn(hidden_nodes,visible_nodes) # W
    self.hidden_bias = torch.randn(1, hidden_nodes) # a
    self.visible_bias = torch.randn(1, visible_nodes ) # b

  # wx + b (hidden nodes)
  def sample_hidden(self, x): # Probabilidad de que el nodo oculto se active
    wx = torch.mm(x, self.weights.t()) # Multiplicacion de x por lamatrix de pesos
    activation = wx + self.hidden_bias.expand_as(wx) # Tendra el mismo tamaño que wx
    p_h_given_v = torch.sigmoid(activation) # probabilidad de que se active el nodo oculto conociendo el nodo visible

    return p_h_given_v, torch.bernoulli(p_h_given_v) # Muestreo de las neuronas ocultas activadas o desactivadas

  # wx + b (visible nodes)
  def sample_visible(self, y): # Probabilidad de que el nodo oculto se active
    wy = torch.mm(y, self.weights) # Multiplicacion de x por lamatrix de pesos
    activation = wy + self.visible_bias.expand_as(wy) # Tendra el mismo tamaño que wx
    p_v_given_h = torch.sigmoid(activation) # probabilidad de que se active el nodo visible conociendo el nodo oculto

    return p_v_given_h, torch.bernoulli(p_v_given_h) # Muestreo de las neuronas ocultas activadas o desactivadas

  # Divergencia contrastante ( Articulo )
  def train(self, v0, vk, ph0, phk):
    self.weights += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.visible_bias += torch.sum((v0 - vk), 0)
    self.hidden_bias += torch.sum((ph0 - phk), 0)




### Restricted Boltzman Machine (RBM)

In [50]:
visible_nodes = len(training_set[0])
hidden_nodes = 100 # 100 caracteristicas que describen a las peliculas
batch_size = 100

rbm = RBM(visible_nodes, hidden_nodes)

In [51]:
# Entrenando la RBM

number_epoch = 10
for epoch in range(1, number_epoch + 1):
  training_loss = 0
  s = 0. # Contador de observaciones
  for id_user in range(0, number_users - batch_size, batch_size): # Tomando lotes de 100 en 100
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0, _ = rbm.sample_hidden(v0) # Probabilidad de activar un nodeo oculto a partir de las valoraciones conocidas
    for k in range(10): # Iteracion para la divergencia contrastante
      _, hk = rbm.sample_hidden(vk) # Nodos ocultos activados en el paso k-esimo de la divergencia contrastante
      _, vk = rbm.sample_visible(hk)
      vk[v0 < 0] = v0[v0 < 0] # Se mantienen los valores negativos en la fase de entreanmiento
    
    phk, _ = rbm.sample_hidden(vk)
    rbm.train(v0, vk, ph0, phk)
    training_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.

  print("Epoch: " + str(epoch) + ", Loss: " + str(training_loss/s) )


Epoch: 1, Loss: tensor(0.3622)
Epoch: 2, Loss: tensor(0.2517)
Epoch: 3, Loss: tensor(0.2378)
Epoch: 4, Loss: tensor(0.2487)
Epoch: 5, Loss: tensor(0.2468)
Epoch: 6, Loss: tensor(0.2491)
Epoch: 7, Loss: tensor(0.2472)
Epoch: 8, Loss: tensor(0.2445)
Epoch: 9, Loss: tensor(0.2484)
Epoch: 10, Loss: tensor(0.2447)


### Evaluar la RBM

In [52]:
testing_loss = 0
s = 0.
for id_user in range(number_users): # Tomando lotes de 1 en 1
  v = test_set[id_user : id_user + 1]
  vt = test_set[id_user : id_user + 1]
  if len(vt[vt >= 0]) > 0: # Usuarios que han visto almenos una pelicula
    _, h = rbm.sample_hidden(v) # Nodos ocultos activados en el paso k-esimo de la divergencia contrastante
    _, v = rbm.sample_visible(h)
  
    testing_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
    s += 1.
    print("Testing Loss: " + str(testing_loss/s) )

Testing Loss: tensor(0.2353)
Testing Loss: tensor(0.2995)
Testing Loss: tensor(0.3150)
Testing Loss: tensor(0.3113)
Testing Loss: tensor(0.3276)
Testing Loss: tensor(0.2994)
Testing Loss: tensor(0.2837)
Testing Loss: tensor(0.2698)
Testing Loss: tensor(0.2398)
Testing Loss: tensor(0.2280)
Testing Loss: tensor(0.2348)
Testing Loss: tensor(0.2185)
Testing Loss: tensor(0.2242)
Testing Loss: tensor(0.2220)
Testing Loss: tensor(0.2359)
Testing Loss: tensor(0.2318)
Testing Loss: tensor(0.2377)
Testing Loss: tensor(0.2321)
Testing Loss: tensor(0.2356)
Testing Loss: tensor(0.2466)
Testing Loss: tensor(0.2541)
Testing Loss: tensor(0.2535)
Testing Loss: tensor(0.2501)
Testing Loss: tensor(0.2505)
Testing Loss: tensor(0.2426)
Testing Loss: tensor(0.2446)
Testing Loss: tensor(0.2478)
Testing Loss: tensor(0.2452)
Testing Loss: tensor(0.2530)
Testing Loss: tensor(0.2612)
Testing Loss: tensor(0.2603)
Testing Loss: tensor(0.2636)
Testing Loss: tensor(0.2678)
Testing Loss: tensor(0.2717)
Testing Loss: 